In [1]:
# https://www.tensorflow.org/lite/tutorials/model_maker_image_classification
from __future__ import absolute_import, division
from BashColors import C
from CV2_Utils_2 import *
from TarfileFunctions import *

from time import perf_counter, sleep
import itertools, json, os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'

import numpy as np
import cv2
import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

# from tflite_support.metadata_writers import image_classifier
# from tflite_support.metadata_writers import writer_utils
import matplotlib.pyplot as plt

contentPath=os.getcwd()
image_path=os.path.join(contentPath, 'images')
generatorPath=os.path.join(contentPath, 'DataGenerator')
imagePath=join(contentPath, '3b7d7d8a64.jpg')
labelsPath=join(contentPath, 'labels.txt')

initialGlobList:list
with open("initialGlobList.json", 'r') as f:
    initialGlobList = json.load(f)
    
modelDictionary={}
def listNewFiles(initial=initialGlobList, delete=False):
    currentFilesGlob=glob.glob('**')
    if len(initial) == len(currentFilesGlob):
        print(f'{C.BIRed}No new files in content.')
    for fil in currentFilesGlob:
        if not fil in initial:
            if isdir(fil):
                print(f'{C.BIBlue}{fil}')
                if delete:
                    shutil.rmtree(fil)
            elif isfile(fil):
                fullPath=abspath(fil)
                print(f'{C.ColorOff}{fil}')
                print(f'{C.ColorOff}{fullPath}\n')
                # modelDictionary.update(str(fil), str(fullPath))
                if delete:
                    os.remove(fil)
listNewFiles()

Defcon4_mobilenet_v2_float16.tflite
/home/jovyan/Defcon4_mobilenet_v2_float16.tflite

Untitled3.ipynb
/home/jovyan/Untitled3.ipynb

labels.txt
/home/jovyan/labels.txt

Defcon4_mobilenet_v2_dynamic.tflite
/home/jovyan/Defcon4_mobilenet_v2_dynamic.tflite



In [3]:
def convert(img, target_type_min, target_type_max, target_type):
    imin = img.min()
    imax = img.max()

    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b).astype(target_type)
    return new_img

imgu8 = convert(img16u, 0, 255, np.uint8)

NameError: name 'img16u' is not defined